Базовый эксперимент с RNN, с вариационным выводом

В качестве априоного рассматривается распредедение $\mathcal{N}(\mathbf{0}, 1000\mathbf{I})$,
в качестве вариационного --- $\mathcal{N}(\mathbf{m}, \beta \mathbf{I})$.

In [1]:
import numpy as np
import theano
import theano.tensor as T
from sklearn.metrics import f1_score
from collections import OrderedDict
if theano.config.device == 'gpu':
    from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
else:
    from theano.tensor.shared_randomstreams import RandomStreams

Делим выборку в отношении 85%/15%.

In [2]:
X = np.load('semeval_x.npy')
Y= np.load('semeval_y.npy').astype(np.int32)

part = int(len(Y)*0.85)

max_len = max([len(x) for x in X])
X_masked1,X_masked2  = np.zeros((Y.shape[0], max_len, 50)).astype(theano.config.floatX),np.zeros((Y.shape[0], max_len, 50)).astype(theano.config.floatX)
Mask1, Mask2 = np.zeros((Y.shape[0], max_len)).astype(theano.config.floatX), np.zeros((Y.shape[0], max_len)).astype(theano.config.floatX)
for i in xrange(X.shape[0]/2):
    X_masked1[i, :len(X[2*i])] = X[2*i]
    X_masked2[i, :len(X[2*i+1])] = X[2*i+1]
    Mask1[i,  :len(X[2*i])] = 1.0
    Mask2[i,  :len(X[2*i+1])] = 1.0
    
X_train1, X_test1=X_masked1[:part],  X_masked1[part:],
X_train2, X_test2 = X_masked2[:part],X_masked2[part:]

M_train1, M_test1= Mask1[:part], Mask1[part:],
M_train2, M_test2  = Mask2[:part],Mask2[part:]
Y_train,  Y_test = Y[:part], Y[part:]



In [146]:
lr = theano.shared(0.01)

prior_log_sigma = theano.shared(np.log(1000.0))
prior_mu = theano.shared(0.0)
var_log_sigma = theano.shared(np.log(0.01))


batch_size = 25
hidden_size = 50
input_size = 50
epoch_num = 1000

Создание входных тензоров.

Для каждой группы параметров из модели создаем переменную <название группы>_mean. В терминах Graves - наше вариационное распределение выглядит как:
Q([W_mean, U_mean, b_mean, ...], $e^\text{var_log_sigma}$)

Использовани логарифма в отклонении помогает избежать возможных переполнений и пр.

In [148]:
X_tensor1, X_tensor2 = T.tensor3(), T.tensor3()
Mask_matrix1, Mask_matrix2 = T.matrix(), T.matrix()
Y_vector = T.ivector()


W_mean = theano.shared((np.zeros((input_size, hidden_size))).astype(theano.config.floatX))
U_mean = theano.shared((np.zeros((hidden_size, hidden_size)).astype(theano.config.floatX)))
b_mean = theano.shared((np.zeros((hidden_size)).astype(theano.config.floatX)))
h0_mean = theano.shared((np.zeros((hidden_size)).astype(theano.config.floatX)))
softmax_W_mean = theano.shared((np.zeros((2*hidden_size,6)).astype(theano.config.floatX)))
softmax_b_mean = theano.shared((np.zeros(6).astype(theano.config.floatX)))




Сэмплируем параметры.

Теперь все наши параметры имеют на одну размерность больше (размер батча $\times$ старые размерности).
С точки зрения технической реализации, единственное, что нам придется изменить --- это заменить перемножение матриц T.dot на перемножение тензоров T.batched_dot

In [149]:
srng = RandomStreams()
W = srng.normal((X_tensor1.shape[0],input_size, hidden_size))*T.exp(var_log_sigma)   + W_mean
U = srng.normal((X_tensor1.shape[0],hidden_size, hidden_size))*T.exp(var_log_sigma)   + U_mean
b = srng.normal((X_tensor1.shape[0], hidden_size))*T.exp(var_log_sigma)   + b_mean
h0 = srng.normal((X_tensor1.shape[0],  hidden_size))*T.exp(var_log_sigma)   + h0_mean
softmax_W = srng.normal((X_tensor1.shape[0],2*hidden_size,6))*T.exp(var_log_sigma)   + softmax_W_mean
softmax_b = srng.normal((X_tensor1.shape[0],6))*T.exp(var_log_sigma)   + softmax_b_mean

Один шаг кодирования:
    $$h_i = \text{tanh}(\mathbf{XW} + \mathbf{h_{i-1}U} + \mathbf{b}) $$
    
XW уже перемножено для оптимизации.

In [150]:
def encode_step(XW_matrix, hidden):    
    return T.tanh(XW_matrix  + T.batched_dot(hidden, U) + b) 

Обертка кодирования с учетом маски.

In [151]:
def masked_step( XW, hidden,   mask):

        hid_new = encode_step(XW, hidden)
        mask_axis = T.tile(mask, (XW.shape[1], 1)).T
        hid_out = T.switch(mask_axis, hid_new, hidden)
        return hid_out

In [153]:
train_results = [] #test_loss

Построение модели.

In [154]:
XW1 = T.batched_dot(X_tensor1, W)
XW2 = T.batched_dot(X_tensor2, W)

hiddens1, hiddens2 = [h0], [h0]
for i in xrange(max_len):
    hiddens1.append(masked_step(XW1[:,i,:], hiddens1[-1],  Mask_matrix1[:,i]))
    hiddens2.append(masked_step(XW2[:,i,:], hiddens2[-1],  Mask_matrix2[:,i]))
output = T.nnet.softmax(T.batched_dot(T.concatenate([hiddens1[-1], hiddens2[-1]], axis=1 ), softmax_W) + softmax_b)


In [155]:
softmax_cost = -T.sum(T.log(output)[T.arange(Y_vector.shape[0]), Y_vector])*X_train1.shape[0]/X_tensor1.shape[0]



расстояние KL для двух гауссовых распределений.

Смотри русскую вики, в их терминах нулевой индекс соответствует вариационному распределению,
первый индекс - априорному

In [156]:
all_param_mean = [W_mean.flatten(),U_mean.flatten(), h0_mean.flatten(), b_mean.flatten(),softmax_W_mean.flatten(),softmax_b_mean.flatten()]
all_param_tensor = T.concatenate(all_param_mean)
first_part = T.exp(2*var_log_sigma)/T.exp(2*prior_log_sigma)
second_part = T.dot(prior_mu- all_param_tensor,(prior_mu-all_param_tensor).T)/T.exp(2*prior_log_sigma)
third_part = -np.sum([len(i.eval()) for i in all_param_mean])
fourth_part = 2*prior_log_sigma - 2*var_log_sigma 
KLD = 0.5* (first_part + second_part + third_part + fourth_part)

In [157]:
cost = softmax_cost  + KLD


Код adagrad из lasagne

In [158]:
def lasagne_adagrad(loss, params, learning_rate=1.0, epsilon=1e-6):   
    grads = [T.grad(loss,p) for p in params]
    updates = OrderedDict()

    for param, grad in zip(params, grads):
        value = param.get_value(borrow=True)
        accu = theano.shared(np.zeros(value.shape, dtype=value.dtype),
                             broadcastable=param.broadcastable)
        accu_new = accu + grad ** 2
        updates[accu] = accu_new
        updates[param] = param - (learning_rate * grad /
                                  T.sqrt(accu_new + epsilon))

    return updates

Компиляция

In [159]:
params_to_optimize  = [W_mean,U_mean,b_mean,softmax_W_mean,softmax_b_mean,h0_mean, var_log_sigma]
train_fn = theano.function([X_tensor1, X_tensor2, Mask_matrix1, Mask_matrix2,  Y_vector], cost,  
                           updates=lasagne_adagrad(cost, params_to_optimize, learning_rate=lr), 
                           on_unused_input='ignore' )
predict_fn = theano.function([X_tensor1, X_tensor2, Mask_matrix1, Mask_matrix2], output, on_unused_input='ignore')


Валидация будет происходить на каждой десятой эпохе.
Выбирается случайная сбалансированная подвыборка из  тестовой выборки и считается их macro-мера.

In [160]:
from random import sample
def my_eval():
    print 'validation'    
    min_class = min([len(np.where(Y_test==i)[0]) for i in range(6)])
    results = []
    for _ in xrange(10):
        idx = []
        for i in range(6):
            class_idx = np.where(Y_test==i)[0].tolist()
            idx.extend(sample(class_idx, min_class))
        pred = np.argmax(predict_fn(X_test1[idx], X_test2[idx], M_test1[idx], M_test2[idx]), axis=1)
        results.append(f1_score(Y_test[idx], pred, average='macro'))
    print np.mean(results), np.std(results)
    train_results.append(results)
    
    

Непосредственно запуск оптимизации.
Каждую эпоху смотрим среднее по батчам значение Evidence. В принципе, на основе этого показателя можно сделать раннюю остановку.


In [161]:
for epoch in xrange(1000):
    print 'Epoch', epoch
    batch_results = []
    for batch_start in xrange(0, X_train1.shape[0] + batch_size,  batch_size):
        batch_ids = range(batch_start, min(X_train1.shape[0], batch_start+batch_size))
        if len(batch_ids)==0:
            break
        x1 = X_train1[batch_ids]
        x2 = X_train2[batch_ids]
        m1 = M_train1[batch_ids]
        m2 = M_train2[batch_ids]
        y = Y_train[batch_ids]        
        batch_results.append( train_fn(x1,x2, m1, m2, y))
    print np.mean(batch_results) 
    if epoch%10==0:
        y_pred = np.argmax(predict_fn(X_train1, X_train2, M_train1, M_train2), axis=1)
        print f1_score(Y_train, y_pred, average='macro')
        my_eval()
    

Epoch 0
7751.01861109
0.261316251838
validation
0.207374459576 0.00904126180089
Epoch 1
7368.20348426
Epoch 2
7288.18688508
Epoch 3
7218.33364078
Epoch 4
7200.08060743
Epoch 5
7165.36822853
Epoch 6
7144.24684106
Epoch 7
7129.23660407
Epoch 8
7099.82774857
Epoch 9
7090.72143362
Epoch 10
7070.66637171
0.33356965524
validation
0.291289155372 0.00983839797782
Epoch 11
7052.12288709
Epoch 12
7040.60261961
Epoch 13
7022.00553541
Epoch 14
7001.25349624
Epoch 15
6995.40918029
Epoch 16
6979.34943259
Epoch 17
6959.77400488
Epoch 18
6946.39785854
Epoch 19
6944.00406667
Epoch 20
6923.17446163
0.350917703652
validation
0.314468172979 0.0126526550972
Epoch 21
6915.48668116
Epoch 22
6904.05281996
Epoch 23
6895.41382684
Epoch 24
6886.95656902
Epoch 25
6861.25562497
Epoch 26
6859.43768698
Epoch 27
6851.93125793
Epoch 28
6846.24949813
Epoch 29
6837.16604334
Epoch 30
6841.20226376
0.363514328423
validation
0.317907470203 0.010821488463
Epoch 31
6818.49318695
Epoch 32
6811.35424016
Epoch 33
6808.49638589


KeyboardInterrupt: 